In [1]:
import torch
import os
from pathlib import Path
from datetime import datetime
import wandb

# 본 과제 제출자는 현재 우분투 도커 환경에서 작업중이므로 다음과 같이 경로 설정
BASE_PATH="/home/Deep-Learning-study"
import sys
sys.path.append(BASE_PATH)

CURRENT_FILE_PATH = os.getcwd()
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")

if not os.path.isdir(CHECKPOINT_FILE_PATH):
  os.makedirs(os.path.join(CURRENT_FILE_PATH, "checkpoints"))

In [2]:
from _01_code._15_lstm_and_its_application.f_arg_parser import get_parser
from _01_code._15_lstm_and_its_application.g_crypto_currency_regression_train_lstm import get_btc_krw_data
from _01_code._15_lstm_and_its_application.i_crypto_currency_classification_train_lstm import get_model

In [3]:
def test(test_model):
    # 테스트용 데이터로더만 가져옴 (분류 태스크용)
    _, _, test_data_loader = get_btc_krw_data(is_regression=False)

    # 모델을 평가 모드로 설정 (dropout, batch norm 등이 평가 모드로 변경됨)
    test_model.eval()

    # 정확도 계산을 위한 변수 초기화
    num_corrects_test = 0      # 정확히 예측한 샘플 수
    num_tested_samples = 0     # 전체 테스트 샘플 수

    print("[TEST DATA]")
    # gradient 계산 비활성화 (메모리 사용량 감소, 연산 속도 향상)
    with torch.no_grad():
        # 테스트 데이터의 배치를 하나씩 처리
        for test_batch in test_data_loader:
            # 입력 데이터와 정답 레이블을 배치에서 추출
            input_test, target_test = test_batch

            # 모델을 통해 예측값 계산
            output_test = test_model(input_test)

            # 가장 높은 확률을 가진 클래스를 예측값으로 선택
            predicted_test = torch.argmax(output_test, dim=1)
            # 정답과 예측이 일치하는 개수 누적
            num_corrects_test += torch.sum(torch.eq(predicted_test, target_test))

            # 처리된 샘플 수 누적
            num_tested_samples += len(input_test)

        # 전체 정확도 계산 (백분율)
        test_accuracy = 100.0 * num_corrects_test / num_tested_samples

        # 전체 테스트 정확도 출력
        print(f"TEST RESULTS: {test_accuracy:6.3f}%")

        # 각 샘플별 예측 결과 출력
        for idx, (output, target) in enumerate(zip(output_test, target_test)):
            # 결과 출력 포맷:
            # 인덱스: 예측 클래스 <--> 실제 클래스
            print("{0:2}: {1:6,.2f} <--> {2:6,.2f}".format(
                idx,                            # 데이터 포인트 인덱스
                torch.argmax(output).item(),    # 예측한 클래스 (0: 하락, 1: 상승)
                target.item()                   # 실제 클래스
            ))

In [6]:
def main(args):
    # 현재 시간을 문자열로 변환 (실행 식별자로 사용)
    run_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    # wandb 설정을 위한 하이퍼파라미터 딕셔너리 생성
    config = {
        'epochs': args.epochs,                    # 총 학습 에폭 수
        'batch_size': args.batch_size,           # 미니배치 크기
        'validation_intervals': args.validation_intervals,  # 검증 수행 주기
        'learning_rate': args.learning_rate,      # 학습률
        'early_stop_patience': args.early_stop_patience,  # 조기 종료 인내 횟수
        'early_stop_delta': args.early_stop_delta,  # 조기 종료 임계값
    }

    # wandb 프로젝트 초기화 (테스트 모드)
    project_name = "lstm_classification_btc_krw"
    wandb.init(
        mode="disabled",                          # 테스트 시에는 wandb 비활성화
        project=project_name,                     # 프로젝트 이름
        notes="btc_krw experiment with lstm",     # 실험 설명
        tags=["lstm", "regression", "btc_krw"],   # 관련 태그
        name=run_time_str,                       # 실행 이름 (시간 기반)
        config=config                            # 설정값들
    )

    # 분류 모델 인스턴스 생성
    test_model = get_model()

    # 저장된 최신 모델 파일 경로 설정
    latest_file_path = os.path.join(
        CHECKPOINT_FILE_PATH, 
        f"{project_name}_checkpoint_2024-12-07_07-46-08.pt"    # 최신 체크포인트 파일명
    )
    print("MODEL FILE: {0}".format(latest_file_path))
    
    # 저장된 모델의 가중치를 CPU에 로드
    test_model.load_state_dict(
        torch.load(latest_file_path, map_location=torch.device('cpu'))
    )

    # 테스트 수행
    test(test_model)

In [7]:
if __name__ == "__main__":
    import sys
    if 'ipykernel' in sys.modules:  # Jupyter Notebook에서 실행 중인지 확인
        # Jupyter에서 실행할 때는 기본값 사용
        class Args:
            def __init__(self):
                self.wandb = True
                self.batch_size = 32
                self.epochs = 100
                self.learning_rate = 0.001
                self.weight_decay = 0.0001
                self.validation_intervals = 1
                self.early_stop_patience = 5
                self.early_stop_delta = 0.001
        args = Args()
    else:
        # 일반 Python 스크립트로 실행할 때는 argparse 사용
        parser = get_parser()
        args = parser.parse_args()
    
    main(args)

MODEL FILE: /home/Deep-Learning-study/_02_homeworks/hw4/checkpoints/lstm_classification_btc_krw_checkpoint_2024-12-07_07-46-08.pt


/tmp/ipykernel_114453/1585746600.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(latest_file_path, map_location=torch.device('cpu'))


[TEST DATA]
TEST RESULTS: 53.333%
 0:   1.00 <-->   1.00
 1:   1.00 <-->   0.00
 2:   1.00 <-->   0.00
 3:   1.00 <-->   1.00
 4:   1.00 <-->   0.00
 5:   1.00 <-->   1.00
 6:   1.00 <-->   0.00
 7:   1.00 <-->   0.00
 8:   1.00 <-->   1.00
 9:   1.00 <-->   1.00
10:   1.00 <-->   0.00
11:   1.00 <-->   1.00
12:   1.00 <-->   1.00
13:   1.00 <-->   0.00
14:   1.00 <-->   1.00
15:   1.00 <-->   1.00
16:   1.00 <-->   0.00
17:   1.00 <-->   1.00
18:   1.00 <-->   1.00
19:   1.00 <-->   0.00
20:   1.00 <-->   1.00
21:   1.00 <-->   0.00
22:   1.00 <-->   1.00
23:   1.00 <-->   1.00
24:   1.00 <-->   0.00
25:   1.00 <-->   0.00
26:   1.00 <-->   0.00
27:   1.00 <-->   0.00
28:   1.00 <-->   1.00
29:   1.00 <-->   1.00
